## Step 1
HTML Obtain

In [17]:
import requests
from bs4 import BeautifulSoup
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
}

url = "https://www.amazon.com/s?k=mobile+phones"
response = requests.get(url, headers=headers)

print(response.status_code)


200


## Step 2
Parsing The HTML

In [37]:
from bs4 import BeautifulSoup

html = response.content
parser = 'lxml'

soup = BeautifulSoup(html, parser)
type(soup)

bs4.BeautifulSoup

In [25]:
web = requests.get("https://www.amazon.com/s?k=mobile+phones&rh=n%3A21514055011%2Cp_123%3A46655&dc&qid=1750948998&rnid=85457740011&ref=sr_nr_p_123_1&ds=v1%3AQDWUlyY3NsiYZY6YKsuUeB4SkeCHqGAR%2FRjqoCoJV%2B8")
soup = BeautifulSoup(web.content,"html.parser")

In [27]:
print(soup.prettify())

<!--
        To discuss automated access to Amazon data please contact api-services-support@amazon.com.
        For information about migrating to our APIs refer to our Marketplace APIs at https://developer.amazonservices.com/ref=rm_5_sv, or our Product Advertising API at https://affiliate-program.amazon.com/gp/advertising/api/detail/main.html/ref=rm_5_ac for advertising use cases.
-->
<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <title>
   Sorry! Something went wrong!
  </title>
  <style>
   html, body {
    padding: 0;
    margin: 0
  }

  img {
    border: 0
  }

  #a {
    background: #232f3e;
    padding: 11px 11px 11px 192px
  }

  #b {
    position: absolute;
    left: 22px;
    top: 12px
  }

  #c {
    position: relative;
    max-width: 800px;
    padding: 0 40px 0 0
  }

  #e, #f {
    height: 35px;
    border: 0;
   

## Step 3
Data Extraction

In [69]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

# Setup headless Chrome
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Function to extract product links from listing page
def get_product_links(page_number):
    url = f"https://www.amazon.com/s?k=mobile+phones&page={page_number}"
    driver.get(url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    links = []
    for a in soup.select("a.a-link-normal.s-no-outline"):
        href = a.get('href')
        if href and '/dp/' in href:
            full_link = "https://www.amazon.com" + href.split('?')[0]
            if full_link not in links:
                links.append(full_link)
    return links

# Function to get detailed product info
def extract_product_info(url):
    driver.get(url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    def get_detail(label):
        tag = soup.find('span', string=label)
        if tag:
            value = tag.find_next('span', class_='a-size-base po-break-word')
            return value.get_text(strip=True) if value else None
        return None

    # Title
    title_tag = soup.find('span', id='productTitle')
    title = title_tag.get_text(strip=True) if title_tag else None

    # Price
    price_tag = soup.find('span', class_='a-offscreen')
    price = price_tag.get_text(strip=True) if price_tag else None

    return {
        "Product Title": title,
        "Price": price,
        "Brand": get_detail("Brand"),
        "Operating System": get_detail("Operating System"),
        "RAM": get_detail("Ram Memory Installed Size"),
        "CPU Model": get_detail("CPU Model"),
        "CPU Speed": get_detail("CPU Speed"),
        "Memory Storage": get_detail("Memory Storage Capacity"),
        "Screen Size": get_detail("Screen Size"),
        "Resolution": get_detail("Resolution"),
        "Model Name": get_detail("Model Name"),
        "Wireless Carrier": get_detail("Wireless Carrier"),
        "URL": url
    }

# Main execution
all_data = []

for page in range(1, 21): 
    print(f"📄 Scraping page {page}...")
    product_links = get_product_links(page)
    for link in product_links:
        try:
            data = extract_product_info(link)
            all_data.append(data)
            print("✅", data["Product Title"])
        except Exception as e:
            print("❌ Error:", link, str(e))

driver.quit()

# Save to CSV
df = pd.DataFrame(all_data)
df.to_csv("amazon_phones.csv", index=False)
print("✔️ Done. Data saved to amazon_phones.csv")


📄 Scraping page 1...
✅ Samsung Galaxy A16 5G A Series, Unlocked Android Smartphone, Large AMOLED Display, Durable Design, Super Fast Charging, Expandable Storage, 5G Connectivity, US Version, 2025, Blue Black
✅ Samsung Galaxy S25 Ultra, 512GB Smartphone, Unlocked Android, AI Night Mode Camera, Snapdragon 8 Elite Fast Processor, 5000mAh Battery, Built-in S Pen, 2025, US 1 Yr Warranty, Titanium Black
✅ Motorola Moto G 5G | 2024 | Unlocked | Made for US 4/128GB | 50MP Camera | Sage Green
✅ Tracfone | Motorola Moto g Play 2024 | Locked | 64GB | 5000mAh Battery | 50MP Quad Pixel Camera | 6.5-in. HD+ 90Hz Display | Sapphire Blue
✅ Samsung Galaxy A36 5G A Series, 128GB Unlocked Android Smartphone, AMOLED Display, Rugged Design, 5G Connectivity, Affordable Performance, US Version, 2025, US 1 Yr Warranty, Awesome Lavender
✅ Reno 11 Pro 8GB+256GB Unlocked Android Phone 50MP+13MP Unlocked Cell Phone 6000 mAh Large Battery Camera Smartphone 4G Dual SIM (Jade Green)
✅ TCL 60 XE NXTPAPER 5G Cell Pho

In [75]:
df=pd.read_csv("amazon_phones.csv")
df.head()

,Product Title,Price,Brand,Operating System,RAM,CPU Model,CPU Speed,Memory Storage,Screen Size,Resolution,Model Name,Wireless Carrier,URL
0,"Samsung Galaxy A16 5G A Series, Unlocked Andro...",$174.99,Samsung,"Android 14, One UI",4 GB,Others,2.4 GHz,128 GB,6.7 Inches,1880 x 2340,Galaxy A16,Unlocked for All Carriers,https://www.amazon.com/SAMSUNG-Unlocked-Smartp...
1,"Samsung Galaxy S25 Ultra, 512GB Smartphone, Un...","$1,219.99",Samsung,"One UI 7, Android 15",12 GB,Snapdragon,3.5 GHz,512 GB,6.9 Inches,2560 x 1440,Samsung Galaxy S25 Ultra,Unlocked for All Carriers,https://www.amazon.com/Smartphone-Unlocked-Pro...
2,Motorola Moto G 5G | 2024 | Unlocked | Made fo...,$163.83,Motorola,Android 14,4 GB,Others,"1.8, 2.0 GHz",128 GB,6.6 Inches,1612x720,Moto G 5G 2024,NaN,https://www.amazon.com/Moto-Unlocked-128GB-Cam...
3,Tracfone | Motorola Moto g Play 2024 | Locked ...,$39.88,TracFone,Android 13.0,4 GB,NaN,NaN,4 GB,6.53 Inches,1600 x 720 pixels,Moto G Play,Tracfone,https://www.amazon.com/Tracfone-Motorola-5000m...
4,"Samsung Galaxy A36 5G A Series, 128GB Unlocked...",$391.83,Samsung,"ONE UI 6.0, Android 14",6 GB,Snapdragon,NaN,128 GB,6.6 Inches,NaN,Galaxy A36,Unlocked for All Carriers,https://www.amazon.com/A36-Unlocked-Smartphone...


Size of Dataset

In [77]:

df.shape

(306, 13)